In [1]:
import numpy as np 
import scipy.stats as ss
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
def YTM (pv: float, maturity: int, cashflow: float = 100):
    """
    This function calculates the yield-to-maturity (ytm) of the promised cash flow
    :param pv: the promised cash flow of the annuity where the calculated value of this promised cash flow
    :param cashflow: the provided cashflow per each coupon data
    :param maturity: the maturity of the provided fixed income asset in months (e.g. 3yr = 36 months)
    return: The ytm is calculated as a continuously compounding annual rate
    """
    
    ytm = 0.0
    step = 0.01

    # Calculate the ytm to within 0.000001, i.e., 0.0001% or 0.01 bps
    while (step > 0.000001):

        # Keep increasing the trial ytm until the resulting present value (pv0) is too small (less than pv).
        while (True):

            # Calculate the present value with a discount rate of "ytm".
            pv0 = 0
            for m in range(1, maturity+1):
                pv0 += cashflow * np.math.exp(-ytm * m / 12.0)

            # Is ytm now too big? If not, increase it further by the amount "step".
            if (pv0 > pv):
                ytm += step

            # If so, reduce it by "step" and break out of the "while (1)" loop. Reduce the step size for the next iteration.
            else:
                ytm -= step
                step /= 10.0
                break

    #return the yield-to-maturity as a percent.
    return 100.0 * ytm              

In [3]:
credit_rating = {0.035:"AAA", 0.04:"AA", 0.045:"A", 0.05:"BBB", 0.06:"Speculative"}

In [4]:
def cash_waterfall(cashflow:int, credit_ratings:dict):
    # promised cash flow per CDO
    ideal_cash = (100 * 20) / 5 
    
    cash = 0 
    defaults = 0
    
    for i in range(1,6):
        # if the cashflow is greate than the ideal cash flow we allocate the cash and reduce the cash pool
        if cashflow > ideal_cash:
            cash = ideal_cash
            cashflow -= ideal_cash
        else:
            # if you lack sufficient cash, then remaining tranches recieve no money
            if defaults == 0:
                cash = cashflow
                defaults += 1
            else:
                cash = 0
        
        # continually add streams of cash per tranche accordingly
        credit_ratings[str(i)] += cash
    
    return credit_ratings
        

## Monte-Carlo Simulation

In [ ]:
rho = 0.0
tranches = ["1", "2", "3", "4", "5"]

while (rho < 1):
    # forming the N-standard normal and its covariance matrix cov to form the correlated vector normal A
    N = np.random.normal(loc=0, scale=1, size=20)
    cov = np.reshape(np.array([1 if i==j else rho for j in range(20) for i in range(20)]),(20,20)) 
    A = N.dot(np.linalg.cholesky(cov))

    # create the default times with which to modify 
    U = ss.norm.cdf(A, loc=0, scale=1)
    T = -50*np.log(U)

    epv = 0
    maturity = 360
    credit_rating = {"1":0, "2":0, "3":0, "4":0, "5":0}

    # 30 year maturity CDO being tranched
    for m in range(1,maturity+1):
        cash = sum([100 if (i > m/12) else 0 for i in T])
        epv += cash * np.math.exp(-0.03*m/12)
    
        # cash-flow waterfall given the total EPV paid out
        credit_rating = cash_waterfall(cashflow=epv, credit_ratings=credit_rating)
    
    print("At correlation parameter {}".format(round(rho, 2)))
    print('--------------------------------------------------------------------------------')
    for x in tranches:
        val = credit_rating[x]
        print("Our cashflow-distribution for {} CDO is ${}".format(x, round(val,2)))
        print("The yeild to maturity is {}".format(YTM(pv=val,maturity=360,cashflow=100)))
    print('================================================================================')
    
    rho += 0.1

At correlation parameter 0.0
--------------------------------------------------------------------------------
Our cashflow-distribution for 1 CDO is $144000.0
The yeild to maturity is -1.1110999999999998
Our cashflow-distribution for 2 CDO is $144000.0
The yeild to maturity is -1.1110999999999998
Our cashflow-distribution for 3 CDO is $65311.77
The yeild to maturity is -1.1110999999999998
Our cashflow-distribution for 4 CDO is $0
The yeild to maturity is 892841.5281148463
Our cashflow-distribution for 5 CDO is $0
